# 2. Pandas - IO tools

In [ ]:
%pylab
from pandas import Series, DataFrame
import pandas as pd

## 讀寫本文格式的數據

將text轉換為DataFrame的函數，其選項分為:
- 索引
- 類型推斷 和 數據轉換
- 日期解析
- 佚代
- 不規整數據問題

類型推斷(type inference)是最重要的功能之一，不需要指定列的資料型態

In [ ]:
!cat ex1.csv

In [ ]:
!type ex1.csv

In [ ]:
# read_csv 讀入 csv檔案
df = pd.read_csv('ex1.csv')
df

In [ ]:
# 也可以讀入table，不過需要指定分隔符號
df = pd.read_table('ex1.csv', sep = ',')
df

In [ ]:
# 沒有欄位名稱列的檔案
!type ex2.csv

In [ ]:
# 預設會把第一列當作 欄位名稱列
df = pd.read_csv('ex2.csv', )
df

In [ ]:
# 標示沒有欄位名稱列
df = pd.read_csv('ex2.csv', header = None)
df

In [ ]:
# 自定義 欄位名稱
fields = ['a', 'b', 'c', 'd', 'message']
df = pd.read_csv('ex2.csv', names = fields)
df

In [ ]:
# 可以 使用 index_col 參數，將某一欄設定為DataFrame的索引
fields = ['a', 'b', 'c', 'd', 'message']
df = pd.read_csv('ex2.csv', names = fields, index_col = 'message')
df

In [ ]:
# 可以 使用 index_col 參數，將多個欄設定為DataFrame的層次化索引 
!type ex3.csv
df = pd.read_csv('ex3.csv', index_col = ['key1', 'key2'])
df

In [ ]:
# 如果不是以固定的分隔符號來分隔字段，可以用 read_table + regex 作為 sep參數
# 由於列名比資料列的數量少，因此read_table推斷第一列應該是DataFrame的索引
# 以不定數量的空白做分隔
!type "ex3 - 1.csv"  
df = pd.read_table('ex3 - 1.csv', sep = '\s+')
df

In [ ]:
# 讀檔時，可以用 skiprows 來跳過指定的 rows
!type ex4.csv
df = pd.read_csv('ex4.csv', skiprows = [0, 2, 3], index_col = 'message')
df

In [ ]:
# 缺失數據的處理
# read_csv 會自動判斷，然後以NaN標示缺失數據的位置
!type ex5.csv
df = pd.read_csv('ex5.csv', index_col = 'something')
df

In [ ]:
# isnull()，判斷元素是否為NaN
df.isnull()

In [ ]:
pd.isnull(df)

In [ ]:
# na_values 參數可指定用於標示缺失數據的字串
df = pd.read_csv('ex5.csv', index_col = 'something', na_values = ['NULL'])
df

In [ ]:
# 為各列分別指定不同的 缺失值標示字串
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
df = pd.read_csv('ex5.csv', na_values = sentinels)
df

### 逐塊讀取文本文件

In [ ]:
# 設定 nrows參數，設定讀入的列數
!type ex5.csv
df = pd.read_csv('ex5.csv', nrows = 2)
df

In [ ]:
# 如果要逐塊讀取，則設定chunksize
!type ex5.csv
chunker = pd.read_csv('ex5.csv', chunksize = 2)
chunker

In [ ]:
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['something'].value_counts(), fill_value = 0)
tot = tot.sort_values(ascending = False)
tot

## 將數據寫出到文本格式

In [ ]:
!type ex5.csv
df = pd.read_csv('ex5.csv')
df

In [ ]:
# 以 to_csv() 將數據寫出到一個 以逗號分隔 的檔案中
df.to_csv('ex5-1.csv')
!type "ex5-1.csv"

In [ ]:
# 寫出的時候，可以設定 sep 參數 指定其他的分隔符號
df.to_csv('ex5-1.csv', sep = '|')
!type "ex5-1.csv"

In [ ]:
# 設定 na_rep 參數，以其他的符號 明確地標示 缺失值
df.to_csv('ex5-1.csv', na_rep = 'NULL')
!type "ex5-1.csv"

In [ ]:
# 可以禁止列出 row, column的標籤
# 不輸出index、header
df.to_csv('ex5-1.csv', na_rep = 'NULL', index = False, header = False) 
!type "ex5-1.csv"

In [ ]:
# 不輸出index
df.to_csv('ex5-1.csv', na_rep = 'NULL', index = False) 
!type "ex5-1.csv"

In [ ]:
# 設定 cols 參數，只寫出一部分的欄位
df
df.to_csv("ex5-1.csv", index = False, cols = ['a', 'b', 'c']) # 好像無效呢?
!type "ex5-1.csv"

In [ ]:
# Series 也有to_csv方法
dates = pd.date_range('1/1/2000', periods = 7)
dates

In [ ]:
ts = Series(np.arange(7), index = dates)
ts

In [ ]:
# Series物件 也有to_csv方法
ts.to_csv('treseries.csv')
!type "treseries.csv"

In [ ]:
# Series類別 也有to_csv方法 (頂層)
Series.to_csv(ts, 'treseries.csv')
!type "treseries.csv

In [ ]:
# 使用 from_csv 將檔案讀入成為 Series
# 有 date欄位，須設定 parse_dates 參數
ts = Series.from_csv('treseries.csv', parse_dates = True)
ts

## JSON(JavaScript Object Notation)數據

In [ ]:
obj = """
{
"name": "Wes", 
"place_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"}, {"name": "Wei", "age": 25, "pet": "Cisco"}]
}
"""

In [ ]:
# 用 json.loads 可將JSON字串還原成 dict物件
import json

result = json.loads(obj)
result

In [ ]:
# JSON物件其實是 dict 物件
type(result)

In [ ]:
# 使用索引，可以探及 dict內部的資料
type(result['siblings'][0]['age'])

In [ ]:
# json.dumps 可將dict物件轉換成 JSON字串
# json字串 和json物件 需區分清楚
# json物件 其實就是 dict
json.dumps(result)

In [ ]:
result['siblings']

In [ ]:
# 以JSON物件建構DataFrame
df_siblings = DataFrame(result['siblings'], columns = ['age', 'name', 'pet']).T
df_siblings

In [ ]:
# DataFrame有 to_json() 方法，可將DataFrame序列化
siblings_json_string = df_siblings.to_json()
siblings_json_string

In [ ]:
siblings_json = json.loads(siblings_json_string)
siblings_json

In [ ]:
# DataFrame有 from_dict() 方法，可反序列化
df_siblings = DataFrame.from_dict(siblings_json)
df_siblings

## Web訊息收集

[Yahoo股票資料抓取](../%E7%B7%B4%E7%BF%92%20-%20%E8%82%A1%E7%A5%A8%E8%B3%87%E6%96%99%E5%BD%99%E6%95%B4_Yahoo%E8%82%A1%E5%B8%82%20-%20%E5%95%8F%E9%A1%8C.ipynb)

## 二進制數據格式

In [ ]:
# pandas物件都有一個 save方法，可以將物件數據以pickle的形式保存到硬碟
df = pd.read_csv('ex1.csv')
df

In [ ]:
type(df)

In [ ]:
# 輸出 pickle資料到檔案
import pickle
df.to_pickle('ex1.pickle')
df = None
del df

In [ ]:
# 讀入 pickle檔案資料成為物件 
df = pickle.load(open('ex1.pickle', 'rb'))
df

In [ ]:
type(df)

### 讀取 Microsoft Excel文件

In [ ]:
# 使用 ExcelFile 方法
xls_file = pd.ExcelFile('test.xls', header = None)
table = xls_file.parse('Sheet1')
table

In [ ]:
type(table)

## 使用數據庫

In [ ]:
# 使用 SQLite3

import sqlite3

# 連接資料庫
con = sqlite3.connect(':memory:')

# 建立資料表
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);
"""
con.execute(query)
con.commit()

# 插入資料
data = [('Atlanta', 'Georgia', 1.25, 6), 
        ('Tallahassee', 'Florida', 2.6, 3), 
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()


# 查詢資料
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
# cursor.description 包含 欄位資訊
cursor.description

In [ ]:
# 用資料庫的資料建立 DataFrame
df = DataFrame(rows, columns = [f[0] for f in cursor.description])
df

In [ ]:
# 使用 pandas.io.sql 來讀取資料庫資料並創建 DataFrame
import pandas.io.sql as sql
df = sql.read_sql('select * from test', con)
df